In [2]:
#import dependencies 
import pandas as pd 
import requests 
import json 
import snowflake.connector
import config 
import os 
import time 
import numpy as np

In [3]:
#cereate connection to specigic database/schema 
con = snowflake.connector.connect(
    user=config.keys['user'],
    password=config.keys['password'],
    account=config.keys['account'],
    warehouse = 'WAREHOUSE_BLUE',
    database  = 'DATABASE_BLUE',
    schema = 'SCHEMA_BLUE'
)

create_table_query = (
    """create or replace TABLE DATABASE_BLUE.SCHEMA_BLUE.BLUE4 (
	"fiscal_year" VARCHAR(50),
	"payroll_number" VARCHAR(50),
	"agency_name" VARCHAR(50),
	"last_name" VARCHAR(50),
	"first_name" VARCHAR(50),
	"mid_init" VARCHAR(50),
	"agency_start_date" VARCHAR(50),
	"work_location_borough" VARCHAR(50),
	"title_description" VARCHAR(50),
	"leave_status_as_of_june_30" VARCHAR(50),
	"base_salary" VARCHAR(50),
	"pay_basis" VARCHAR(50),
	"regular_hours" VARCHAR(50),
	"regular_gross_paid" VARCHAR(50),
	"ot_hours" VARCHAR(50),
	"total_ot_paid" VARCHAR(50),
	"total_other_pay" VARCHAR(50)
);""" 
) 

# execute
con.cursor().execute(create_table_query)

In [21]:
#get dataset info for funciton 
#get row count for dataset 
r = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json?$select=count(*)")
r.json()

[{'count': '5662713'}]

In [23]:
#get dataset info for funciton 
#get row count for dataset 
data_size = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json?$select=count(*)")
data_size.json()
data_size = data_sample.json()[0]['count']
data_size

'5662713'

In [ ]:
data_sample = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json")

In [ ]:
data_sample = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json")
sample_df = pd.DataFrame(data_sample.json())

In [ ]:
#check for missing column , if misisng then addcolumn and leave null

In [84]:
df = pd.read_csv('temp/2014_data_sample.csv')

"fiscal_year" VARCHAR(50),
	"payroll_number" VARCHAR(50),
	"agency_name" VARCHAR(50),
	"last_name" VARCHAR(50),
	"first_name" VARCHAR(50),
	"mid_init" VARCHAR(50),
	"agency_start_date" VARCHAR(50),
	"work_location_borough" VARCHAR(50),
	"title_description" VARCHAR(50),
	"leave_status_as_of_june_30" VARCHAR(50),
	"base_salary" VARCHAR(50),
	"pay_basis" VARCHAR(50),
	"regular_hours" VARCHAR(50),
	"regular_gross_paid" VARCHAR(50),
	"ot_hours" VARCHAR(50),
	"total_ot_paid" VARCHAR(50),
	"total_other_pay" VARCHAR(50)

In [103]:
df23 = pd.read_csv('temp/2023_data_sample.csv')

In [108]:
df23.columns

Index(['fiscal_year', 'payroll_number', 'agency_name', 'last_name',
       'first_name', 'mid_init', 'agency_start_date', 'work_location_borough',
       'title_description', 'leave_status_as_of_june_30', 'base_salary',
       'pay_basis', 'regular_hours', 'regular_gross_paid', 'ot_hours',
       'total_ot_paid', 'total_other_pay'],
      dtype='object')

In [110]:
len(cols_order)

17

In [67]:
current_cols = ['fiscal_year', 'payroll_number', 'agency_name', 'last_name',
       'first_name', 'mid_init', 'agency_start_date', 'work_location_borough',
       'title_description', 'leave_status_as_of_june_30']

In [112]:
len(cols_order)

17

In [80]:

missing_cols

['base_salary',
 'pay_basis',
 'regular_hours',
 'regular_gross_paid',
 'ot_hours',
 'total_ot_paid',
 'total_other_pay']

In [ ]:
sample_df[match_test]

In [6]:
#get years in dataset
years = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json?$select=distinct fiscal_year")
years = years.json()
years = [i['fiscal_year'] for i in years]
years.sort()

In [7]:
years

['2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023']

In [127]:
yearsr i in years[]: 
    #define function variables 
    url = f"https://data.cityofnewyork.us/resource/k397-673e.json?$limit={300}&$where=fiscal_year={i}"
    #using dataset size as arbitrary limit
    response = requests.get(url)
    
    if response.status_code != 200:
        break 
    
    #save data to temp folder as csv    
    dataframe = pd.DataFrame(response.json())
    
    file_path = f'temp/{i}_data_sample.csv'
    
    #define dataframe format /coloumn order 
    cols_order = ['fiscal_year', 'payroll_number', 'agency_name', 'last_name',
       'first_name', 'mid_init', 'agency_start_date', 'work_location_borough',
       'title_description', 'leave_status_as_of_june_30', 'base_salary',
       'pay_basis', 'regular_hours', 'regular_gross_paid', 'ot_hours',
       'total_ot_paid', 'total_other_pay']
    
    #get current columns and check for #check for missing columns 
    current_cols = dataframe.columns.to_list()
    missing_cols = [i for i in cols_order if i not in current_cols]

    if len(missing_c) > 0: 
        for x in missing_c:
            dataframe[x] = np.nan
    
    print(f'{len(missing_c)} in {i}')    
    
    #re-order columns 
    dataframe = dataframe[['fiscal_year', 'payroll_number', 'agency_name', 'last_name',
       'first_name', 'mid_init', 'agency_start_date', 'work_location_borough',
       'title_description', 'leave_status_as_of_june_30', 'base_salary',
       'pay_basis', 'regular_hours', 'regular_gross_paid', 'ot_hours',
       'total_ot_paid', 'total_other_pay']]

    #save dataframe 
    dataframe.to_csv(file_path, index=False)   
    
    print(len(dataframe.columns))
    print(f'{i}:rows:{len(dataframe)}')
    
    #stage file in snowflake 
    #con.cursor().execute(f'PUT file://{file_path}* @%BLUE4')
    
    #free up memory, clear out variable and delete temp file 
    dataframe = None
    #if os.path.isfile(file_path):
     #   os.remove(file_path)
    
    #
    #time.sleep(10)

16
2014:rows:300
16
2015:rows:300
16
2016:rows:300
16
2017:rows:300
17
2018:rows:300
17
2019:rows:300
17
2020:rows:300
17
2021:rows:300
17
2022:rows:300
17
2023:rows:300


In [130]:
- column number changes 
need to have that column ready to be set as null when no inserts on it 
- need to reorder dataframes before saving to make sure order matches table, AND add blank column if does not exist so it matches fully ?


SyntaxError: invalid syntax (<ipython-input-130-3e06f7808ddd>, line 1)

In [117]:
df16 = pd.read_csv('temp/2015_data_sample.csv')

In [114]:
df17 = pd.read_csv('temp/2017_data_sample.csv')

In [128]:
len(df16.columns.to_list())

16

In [129]:
len(df17.columns.to_list())

16

In [105]:
os.listdir('temp/')

[]

In [8]:
#include "/" at the end of dir path 
def clear_dir(dir_path): 
    for i in os.listdir(dir_path):
        os.remove(f"{dir_path}{i}")
    

In [12]:
clear_dir('temp/')

In [13]:
 for i in os.listdir('temp/'):
        os.remove(f"{'temp/'}{i}")